In [ ]:
# default_exp video

# Video

> API details.

In [ ]:
#export
import cv2, datetime, time, json
from functools import partial
from pathlib import Path
from expoco.screen import *
from expoco.pointer import *
from expoco.keyboard import *

In [ ]:
#export
def _now(): 
    return datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')

In [ ]:
#export
def _new_capture_metadata(stop_after, path, screen_size, video_capture, random_move):
    width, height = [int(video_capture.get(p)) for p in [cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT]]
    assert width >= height
    return dict(count=0, stop_after=stop_after, path=str(path.resolve()), screen_size=screen_size,
                capture_width=width, capture_height=height, trim_width=(width-height)//2, 
                random_move=random_move)

In [ ]:
#export
def _after_key_press(capture_metadata, video_capture, pointer, character):
    "Handle key presses during a video capture session"
    if character == 'C': # Capture
        retval, image = video_capture.read()
        if retval:
            path,screen_size,trim_width=[capture_metadata[k] for k in ['path','screen_size','trim_width']]
#             image = image[:, trim_width:-trim_width] # resize to square
            image = image[100:, 130:-130] # TODO: xxx
            x,y = pointer.get_position()
            assert cv2.imwrite(f'{path}/{_now()}_{x}_{y}.png', image)
            capture_metadata['count'] += 1
            if capture_metadata['count'] == capture_metadata['stop_after']: 
                return 'ESC'
            if capture_metadata['random_move']: 
                pointer.random_move_to(screen_size)
            else:
                pointer.move_to([x + (-10 if x > 100 else 10), y])
                time.sleep(.1)
                pointer.move_to([x, y])
        else:
            print('Nothing read from video capture')
    if character == 'P':
        print('TODO: preview last image captured')
        # and make it possible to discard if it's not right

In [ ]:
#export
def _after_key_press2(capture_metadata, video_capture, pointer, character):
    "Screen sweep on C press"
    if character == 'C': # Capture
#         _pos,_width,_height=[639, 360],3,1
        _pos,_width,_height=[600, 300],50,30
        pointer.move_to(_pos)
        time.sleep(.5)
        for _ in range(10):
            _pos[0]-=1
            _pos[1]-=1
            _width+=2
            _height+=2
            print(_pos,_width,_height)
            def _m(label):
                time.sleep(.03)
                retval, image = video_capture.read()
                if retval:
                    path,screen_size,trim_width=[capture_metadata[k] for k in ['path','screen_size','trim_width']]
                    image = image[100:, 130:-130] # TODO: xxx
                    x,y = pointer.get_position()
                    assert cv2.imwrite(f'{path}/{_now()}_{x}_{y}.png', image)
                    capture_metadata['count'] += 1
                else:
                    print('Nothing read from video capture')
                pointer.move_to(_pos)
            _m('start')
            for i in range(1,_width):
                _pos[0]+=1
                _m('right')
            for i in range(1,_height):
                _pos[1]+=1
                _m('down')
            for i in range(1,_width):
                _pos[0]-=1
                _m('left')
            for i in range(1,_height):
                _pos[1]-=1
                _m('up')

In [ ]:
#export
def capture_session(device_control_functions, expression_id, stop_after=20, random_move=True):
    "Run a video capture session"
    screen,pointer,keyboard=[device_control_functions[k]() for k in ['screen','pointer','keyboard']]
    expression_id = str(expression_id)
    with open('data/config.json') as f: config = json.load(f)
    if expression_id not in config.get('expressions', {}):
        raise Exception(f'{expression_id} is missing from expressions section of data/config.json')
    path = Path(f'data/capture_session_{_now()}_{expression_id}')
    path.mkdir(parents=True, exist_ok=True)
    video_capture = cv2.VideoCapture(0)
    retval, image = video_capture.read() # throw away capture - 1st image is usually poor
    try:
        width, height = [int(video_capture.get(p)) for p in [cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT]]
        assert width >= height
        screen_size = screen.get_screen_size()
        if random_move: pointer.random_move_to(screen_size)
        capture_metadata = _new_capture_metadata(stop_after, path, screen_size, video_capture, random_move)
        print('Ready to capture', config['expressions'][expression_id],
              'with' if random_move else 'without', 'random pointer moves')
        print('Press C to capture')
        keyboard.after_key_press(partial(_after_key_press, 
                                         capture_metadata, video_capture, pointer))
        with open(path/'capture_metadata.json', 'w') as f: json.dump(capture_metadata, f, indent=2)
    finally:
        video_capture.release()

```
from expoco.core import *
capture_session(get_device_control_functions('windows'), 0, 5, False)
```

In [ ]:
# from expoco.core import *
# capture_session(get_device_control_functions('windows'), 0, 5, False)

## TODO:

- make it easy to preview captured images for last session

In [ ]:
# # uncomment and run to clear all captured session data
# import shutil
# for folder_path in Path('data').glob('capture_session*'):
#     shutil.rmtree(folder_path)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 10_screen.ipynb.
Converted 20_pointer.ipynb.
Converted 30_keyboard.ipynb.
Converted 40_video.ipynb.
Converted 90_cli.ipynb.
Converted index.ipynb.
Converted project_lifecycle.ipynb.
